In this notebooks, we'll learn first how to combine multiple DataFrame using the `merge` method and then see how we can further explore the resulting DataFrame.

## Merging _pandas_ DataFrames

In [ ]:
import pandas as pd

In [ ]:
unemployment = pd.read_csv('../data/tn_unemployment.csv')

In [ ]:
unemployment.head()

This unemployment data was obtained from the Burea of Labor Statistics.

Notice that it includes the labor force for each county in the LF column along with the number employed and number unemployed.

We can easily verify the unemployment rate values by dividing the Unemployed column by the LF column. Using `pandas`, we can do these types of operations on whole columns at a time:

In [ ]:
unemployment['Unemployed'] / unemployment['LF']

We can also multiply an entire column by a number. For example, if we can to convert the proportions from above to percentages, we can do

In [ ]:
100 * unemployment['Unemployed'] / unemployment['LF']

Now, let's bring in our second DataFrame.

In [ ]:
population = pd.read_csv('../data/tn_population.csv')

In [ ]:
population.head()

Our goal is to combine the unemployment and population data. In order to do this, _pandas_ needs a common column to join on. 

Notice that the `Name` column from `unemployment` and the `County Name` column from `population` almost work, except that `Name` also includes ", TN". Let's remove this part so that we have a column to join on.

### Brief Detour - String Methods

When working with text data in `pandas`, it is often useful to utilize the built-in sting methods. To use these methods, you must prepend a `.str` before the desired method.

For example, we can make column entirely uppercase using the `upper()` method.

In [ ]:
population['County Name'].str.upper()

Another often useful method is the `replace()` method. To use this method, specify what pattern you want to replace and then the replacement text.

In [ ]:
unemployment['Period'].str.replace('21', '2021')

A method that we can use that will allow us to merge our dataframes is the `.str.split()` method.

Notice that if we split on the comma, the first piece will match what is contained in the `County Name` column.

In [ ]:
unemployment['Name'].str.split(',')

By default, this method returns a list. We can make it return a DataFrame by using the `expand` argument.

In [ ]:
unemployment['Name'].str.split(',', expand = True)

We only want the first column.

In [ ]:
unemployment['Name'].str.split(',', expand = True)[0]

Finally, we can assign this back to the `Name` column.

In [ ]:
unemployment['Name'] = unemployment['Name'].str.split(',', expand = True)[0]

In [ ]:
unemployment.head()

Now, we are ready to merge.

**Option 1:** Use the `left_on` and `right_on` arguments to tell pandas what to merge on.

In [ ]:
pd.merge(left = population, right = unemployment, left_on = 'County Name', right_on = 'Name')

Notice that we end up with a duplicated column (with different names).

**Option 2:** Rename the merging column for one of our DataFrames.

In [ ]:
pd.merge(left = population,
         right = unemployment.rename(columns = {'Name': 'County Name'}))

We can even select out just the columns that we need after the merge:

In [ ]:
pd.merge(left = population,
         right = unemployment[['Name', 'unemployment_rate']].rename(columns = {'Name': 'County Name'}))

Once we're happy with the results, we can save them to a DataFrame.

In [ ]:
tn_counties = pd.merge(left = population,
         right = unemployment[['Name', 'unemployment_rate']].rename(columns = {'Name': 'County Name'}))

Now, let's add in some additional information about these counties - which Grand Division they belong to.

In [ ]:
grand_divisions = pd.read_csv('../data/tn_divisions.csv')

In [ ]:
grand_divisions.head()

**Your Turn:** Merge tn_counties with grand_divisions. Save the results back to `tn_counties`.

In [ ]:
# Your Code Here

## Further Methods of Slicing and Exploration

When working with numeric data, it is often useful to look at some summary statistics. The `describe()` method is an easy way to get a summary of a numeric column.

In [ ]:
tn_counties['Population'].describe()

Let's now find all counties in East Tennessee that have at least 100,000 residents.

We saw last week how we could use `.loc` to filter a DataFrame. If there are multiple conditions we want to filter on, we can still use `.loc`, but we must first surround each condition in parentheses and separate the conditions by an &.

In [ ]:
tn_counties.loc[(tn_counties['Division'] == 'East') & (tn_counties['Population'] >= 100000)]

The & means that pandas will find all rows that meet both conditions. You can think of & as "and". If we want to find all rows that meet either condition a _or_ condition b, we can use the |, which can be thought of as "or".

For example, if we want to find any counties that have either at least 250,000 residents _or_ have unemployment less than 4%, we can accomplish this using the following code.

In [ ]:
tn_counties.loc[(tn_counties['Population'] >= 250000) | (tn_counties['unemployment_rate'] < 4)]

If we want to filter down to the rows matching 2 or more conditions, we can either string together a series of conditions with | or we can use the `.isin` method and provide a list of values we want to match.

For example, if we want to look at the rows for Davidson, Shelby, Knox, and Hamilton Counties, we could accomplish it as follows:

In [ ]:
metro_counties = ['Davidson County', 'Shelby County', 'Knox County', 'Hamilton County']

tn_counties.loc[tn_counties['County Name'].isin(metro_counties)]

Finally, if we want to negate a condition, we can do so using a ~.

For example, let's say we want to find all counties besides the four metro counties above.

In [ ]:
tn_counties.loc[~tn_counties['County Name'].isin(metro_counties)]